In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV,train_test_split
from sklearn.model_selection import cross_validate

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [3]:
all_features = list(range(180)) 
clin_demo_comp = list(range(154,180)) 
clin_demo_cyto_gen_comp = list(range(1,180)) 
comp = list(range(163,180)) 
cyto_comp = list(range(85,154))+list(range(163,180)) 
cyto_gen_comp = list(range(1,154))+list(range(163,180)) 
eln_clin_demo_comp = [0]+list(range(154,180)) 
eln_cyto_comp = [0]+list(range(85,154))+list(range(163,180)) 
eln_cyto_gen_comp = list(range(154))+list(range(163,180)) 
eln_gen_comp = list(range(85))+list(range(163,180)) 
gen_comp = list(range(1,85))+list(range(163,180))
clin_comp = list(range(154,161))+list(range(163,180)) 
clin_cyto_comp = list(range(85,161))+list(range(163,180)) 
clin_gen_comp = list(range(1,85))+list(range(154,161))+list(range(163,180)) 
eln_clin_comp = [0]+list(range(154,161))+list(range(163,180))


#Without age
all_features_without_age = list(range(162))+list(range(163,180))
clin_demo_comp_without_age = list(range(154,162))+list(range(163,180))
clin_demo_cyto_gen_comp_without_age = list(range(1,162))+list(range(163,180))
eln_clin_demo_comp_without_age = [0]+list(range(154,162))+list(range(163,180))
            
            
eln_clin_gen = list(range(85))+list(range(154,161))  
eln_demo_gen = list(range(85))+[161,162] 
eln_clin_demo_cyto_gen =list(range(163)) 
eln_clin_demo_cyto = [0]+list(range(85,163))

eln_clin_demo_gen = list(range(85))+list(range(154,163))
eln_clin_demo = [0] + list(range(154,163))
eln_clin = [0] + list(range(154,161))
eln_cyto_gen = list(range(154))
clin_demo_cyto_gen = list(range(1,163))
clin_demo_cyto = list(range(85,163))
clin_demo_gen = list(range(1,85))+list(range(154,163)) 
clin_demo = list(range(154,163)) 
cyto_gen = list(range(1,154))
cyto = list(range(85,154))
gen = list(range(1,85))
clin_gen = list(range(1,85)) + list(range(154,161))  
clin_cyto = list(range(85,161))  
demo_gen = list(range(1,85)) + [161,162]
demo_cyto = list(range(85,154)) + [161,162]

###Without age:

eln_demo_gen_without_age = list(range(85)) + [161]
eln_clin_demo_cyto_gen_without_age = list(range(162))
eln_clin_demo_cyto_without_age = [0] + list(range(85,162))
eln_clin_demo_gen_without_age = list(range(85)) + list(range(154,162))
eln_clin_demo_without_age = [0] + list(range(154,162))
clin_demo_cyto_gen_without_age = list(range(1,162))
clin_demo_cyto_without_age = list(range(85,162))
clin_demo_gen_without_age = list(range(1,85)) + list(range(154,162)) 
clin_demo_without_age = list(range(154,162))
demo_gen_without_age = list(range(1,85)) + [161]
demo_cyto_without_age = list(range(85,154)) + [161]
gen_age = list(range(1,85)) + [162]
eln=[0]
eln_comp=[0] + list(range(163,180))
eln_age=[0,162]
eln_gen=[0] + list(range(1,85))
eln_cyto=[0] + list(range(85,154))


In [ ]:
df_final = pd.read_table("prognosis_comp_final.tsv")
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5.5,-5,-4.5,-2.5,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)